In [ ]:
import torch
import numpy as np
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
import torchvision.transforms as transforms

# Load mô hình phát hiện khuôn mặt
mtcnn = MTCNN(keep_all=False, post_process=True)

# Load mô hình nhận diện khuôn mặt
model = InceptionResnetV1(pretrained='vggface2').eval()

# Hàm phát hiện khuôn mặt và cắt ảnh
def detect_and_crop_face(image_path):
    image = Image.open(image_path).convert('RGB')
    face = mtcnn(image)  # Phát hiện khuôn mặt

    if face is None:
        print(f"Không tìm thấy khuôn mặt trong {image_path}")
        return None

    return face.unsqueeze(0)  # Thêm batch dimension

# Load và trích xuất embedding cho hai ảnh
img1 = detect_and_crop_face("image1.jpg")
img2 = detect_and_crop_face("image2.jpg")

if img1 is not None and img2 is not None:
    with torch.no_grad():
        emb1 = model(img1)
        emb2 = model(img2)

    # Tính khoảng cách Euclidean
    distance = torch.norm(emb1 - emb2).item()

    # So sánh với ngưỡng
    threshold = 0.8  # Ngưỡng tùy chỉnh
    if distance < threshold:
        print("Hai ảnh có thể là cùng một người.")
    else:
        print("Hai ảnh có thể là hai người khác nhau.")

    print(f"Khoảng cách Euclidean: {distance:.4f}")
